## Import Libraries and Setup Argoverse Stereo Data Loader 

In [1]:
%matplotlib notebook

import copy
import json
import shutil
import os
import time
import math
import random
from PIL import Image
from pathlib import Path
from multiprocessing import Pool

import cv2
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
from PSMNet.models import *
import pandas as pd

from argoverse.data_loading.stereo_dataloader import ArgoverseStereoDataLoader
from argoverse.evaluation.stereo.eval import StereoEvaluator
from argoverse.utils.calibration import get_calibration_config
from argoverse.utils.camera_stats import RECTIFIED_STEREO_CAMERA_LIST

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F

STEREO_FRONT_LEFT_RECT = RECTIFIED_STEREO_CAMERA_LIST[0]
STEREO_FRONT_RIGHT_RECT = RECTIFIED_STEREO_CAMERA_LIST[1]


# Path to the dataset (please change accordingly).
data_dir = "/home/cmiller/Documents/Curtis_Classes/CMPE_249/Project/code/argoverse_stereo_v1.1/"

# Choosing the data split: train, val, or test (note that we do not provide ground truth for the test set).
split_name = "val"

# Choosing a specific log id. For example, 273c1883-673a-36bf-b124-88311b1a80be.
log_ids = os.listdir('./argoverse_stereo_v1.1/rectified_stereo_images_v1.1/val/')

# Creating the Argoverse Stereo data loader.
stereo_data_loader = ArgoverseStereoDataLoader(data_dir, split_name)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Generate Video of Results from Stereo Matching 

In [2]:
# Define Paths to Image Data
DMDirectory = './results/stereo_match_results/70d2aea5-dbeb-333d-b21e-76a7f2f1ba1c/'
ImageDirectory = './argoverse_stereo_v1.1/rectified_stereo_images_v1.1/val/70d2aea5-dbeb-333d-b21e-76a7f2f1ba1c/stereo_front_left_rect/'
ErrorDirectory = './results/stereo_match_error_results/70d2aea5-dbeb-333d-b21e-76a7f2f1ba1c/'

# Get all images from training result directory
images = sorted([image for image in os.listdir(ImageDirectory) if image.endswith(".jpg")])
dms = sorted([image for image in os.listdir(DMDirectory) if image.endswith(".png")])
errors = sorted([image for image in os.listdir(ErrorDirectory) if image.endswith(".png")])

In [3]:
# Read single image to get size of image
image = cv2.imread(os.path.join(ImageDirectory, images[0]))
dm = cv2.imread(os.path.join(DMDirectory, dms[0]))
error = cv2.imread(os.path.join(ErrorDirectory, errors[0]))

# Define size of video frame as 1/4 of combined size
imgHeight, imgWidth, _ = image.shape
dmHeight, dmWidth, _ = dm.shape
errorHeight, errorWidth, _ = error.shape
height = int(imgHeight/4)
width = int((imgWidth + dmWidth + errorWidth)/4)

# Loop through each image and place in video
for i in range(0, len(images)):
    
    # Load images
    image = cv2.imread(os.path.join(ImageDirectory, images[i]))
    dm = cv2.imread(os.path.join(DMDirectory, dms[i]))
    error = cv2.imread(os.path.join(ErrorDirectory, errors[i]))
    
    # Downsample images
    image = cv2.resize(image, (0, 0), None, .25, .25)
    dm = cv2.resize(dm, (0, 0), None, .25, .25)
    error = cv2.resize(error, (0, 0), None, .25, .25)
    
    # Combine images and write to video
    numpy_vertical = np.hstack((image, dm, error))
    video.write(numpy_vertical)

# Close Video
cv2.destroyAllWindows()
video.release()

## Generate Video of Results from PSMNet

In [4]:
# Define Paths to Image Data
DMDirectory = './results/PSMNet_results//70d2aea5-dbeb-333d-b21e-76a7f2f1ba1c/'
ImageDirectory = './argoverse_stereo_v1.1/rectified_stereo_images_v1.1/val/70d2aea5-dbeb-333d-b21e-76a7f2f1ba1c/stereo_front_left_rect/'
ErrorDirectory = './results/PSMNet_error_results//70d2aea5-dbeb-333d-b21e-76a7f2f1ba1c/'

# Get all images from training result directory
images = sorted([image for image in os.listdir(ImageDirectory) if image.endswith(".jpg")])
dms = sorted([image for image in os.listdir(DMDirectory) if image.endswith(".png")])
errors = sorted([image for image in os.listdir(ErrorDirectory) if image.endswith(".png")])

In [5]:
# Read single image to get size of image
image = cv2.imread(os.path.join(ImageDirectory, images[0]))
dm = cv2.imread(os.path.join(DMDirectory, dms[0]))
error = cv2.imread(os.path.join(ErrorDirectory, errors[0]))

# Define size of video frame as 1/4 of combined size
imgHeight, imgWidth, _ = image.shape
dmHeight, dmWidth, _ = dm.shape
errorHeight, errorWidth, _ = error.shape
height = int(imgHeight/4)
width = int((imgWidth + dmWidth + errorWidth)/4)

# Start video 
video = cv2.VideoWriter('./results/PSMNet_results.avi', 0, 3, (width, height))

# Loop through each image and place in video
for i in range(0, len(images)):
    
    # Load images
    image = cv2.imread(os.path.join(ImageDirectory, images[i]))
    dm = cv2.imread(os.path.join(DMDirectory, dms[i]))
    error = cv2.imread(os.path.join(ErrorDirectory, errors[i]))
    
    # Downsample images
    image = cv2.resize(image, (0, 0), None, .25, .25)
    dm = cv2.resize(dm, (0, 0), None, .25, .25)
    error = cv2.resize(error, (0, 0), None, .25, .25)
    
    # Combine images and write to video
    numpy_vertical = np.hstack((image, dm, error))
    video.write(numpy_vertical)

# Close Video
cv2.destroyAllWindows()
video.release()